In [4]:
import csv
from collections import defaultdict

# Путь к файлу
csv_file = 'sales_data.csv'

# Переменные для хранения результатов
total_revenue = 0.0
product_sales = defaultdict(int)
product_revenue = defaultdict(float)

# Ожидаемые столбцы в CSV файле
expected_columns = ['Номер заказа', 'Дата заказа', 'Название товара', 'Категория товара', 'Количество продаж', 'Цена за единицу', 'Общая стоимость']

# Функция быстрой сортировки (Quick Sort)
def quick_sort(arr, key):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x[key] < pivot[key]]
    middle = [x for x in arr if x[key] == pivot[key]]
    right = [x for x in arr if x[key] > pivot[key]]
    return quick_sort(left, key) + middle + quick_sort(right, key)

# Функция пузырьковой сортировки (Bubble Sort)
def bubble_sort(arr, key):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j][key] > arr[j+1][key]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# Функция бинарного поиска
def binary_search(arr, target, key):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid][key] == target:
            return mid
        elif arr[mid][key] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

# Чтение данных из CSV файла
try:
    with open(csv_file, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        # Удаление пробелов из названий столбцов
        reader.fieldnames = [field.strip() for field in reader.fieldnames]
        
        # Проверка наличия всех ожидаемых столбцов
        if not all(column in reader.fieldnames for column in expected_columns):
            missing_columns = [column for column in expected_columns if column not in reader.fieldnames]
            print(f"Ошибка: Отсутствуют столбцы в CSV файле: {', '.join(missing_columns)}")
            exit(1)
        
        for row in reader:
            try:
                # Удаление пробелов из значений в строке
                row = {key.strip(): value.strip() for key, value in row.items()}
                
                # Проверка наличия всех необходимых ключей в строке
                if not all(column in row for column in expected_columns):
                    missing_columns = [column for column in expected_columns if column not in row]
                    print(f"Ошибка: В строке отсутствуют необходимые столбцы: {', '.join(missing_columns)}")
                    continue

                order_number = row['Номер заказа']
                order_date = row['Дата заказа']
                product_name = row['Название товара']
                product_category = row['Категория товара']
                quantity_sold = int(row['Количество продаж'])
                unit_price = float(row['Цена за единицу'])
                total_price = float(row['Общая стоимость'])
                
                # Обновление общих показателей
                total_revenue += total_price
                product_sales[product_name] += quantity_sold
                product_revenue[product_name] += total_price
            
            except ValueError as e:
                print(f"Ошибка преобразования данных в строке: {row} - {e}")
                continue

except FileNotFoundError:
    print(f"Ошибка: Файл {csv_file} не найден.")
    exit(1)
except csv.Error as e:
    print(f"Ошибка при чтении CSV файла: {e}")
    exit(1)

# Проверка наличия данных
if not product_sales or not product_revenue:
    print("Нет данных для анализа.")
    exit(1)

# Подготовка данных для сортировки и поиска
products = [{'name': name, 'sales': sales, 'revenue': product_revenue[name]} for name, sales in product_sales.items()]

# Сортировка товаров по количеству продаж (пузырьковая сортировка)
sorted_by_sales_bubble = bubble_sort(products.copy(), 'sales')

# Сортировка товаров по выручке (быстрая сортировка)
sorted_by_revenue_quick = quick_sort(products.copy(), 'revenue')

# Поиск товара, проданного наибольшее количество раз
best_selling_product = sorted_by_sales_bubble[-1]['name']

# Поиск товара, принесшего наибольшую выручку
top_revenue_product = sorted_by_revenue_quick[-1]['name']

# Формирование отчета
report = {
    'Общая выручка': total_revenue,
    'Товар, проданный наибольшее количество раз': best_selling_product,
    'Количество продаж': product_sales[best_selling_product],
    'Товар, принесший наибольшую выручку': top_revenue_product,
    'Выручка от товара': product_revenue[top_revenue_product],
    'Детали по каждому товару': []
}

for product in products:
    report['Детали по каждому товару'].append({
        'Название товара': product['name'],
        'Количество продаж': product['sales'],
        'Доля в общей выручке': product['revenue'] / total_revenue
    })

# Вывод отчета
print("Отчет по продажам")
print(f"Общая выручка магазина: {report['Общая выручка']:.2f}")
print(f"Товар, проданный наибольшее количество раз: {report['Товар, проданный наибольшее количество раз']} ({report['Количество продаж']} шт.)")
print(f"Товар, принесший наибольшую выручку: {report['Товар, принесший наибольшую выручку']} ({report['Выручка от товара']:.2f})")
print("\nДетали по каждому товару:")
for item in report['Детали по каждому товару']:
    print(f"{item['Название товара']}: {item['Количество продаж']} шт., {item['Доля в общей выручке']:.2%} от общей выручки")


Отчет по продажам
Общая выручка магазина: 920317228.00
Товар, проданный наибольшее количество раз: Lay's Гуакомоле (63455 шт.)
Товар, принесший наибольшую выручку: Микрофон Rode NT1 (454664000.00)

Детали по каждому товару:
Lay's Краб: 1012 шт., 0.01% от общей выручки
Lay's Гуакомоле: 63455 шт., 1.10% от общей выручки
Apple iPhone 14 Pro 256 gb Deep Purple: 27 шт., 0.17% от общей выручки
Apple iPhone 15 256 gb Pink: 52 шт., 0.51% от общей выручки
Macbook Air 13 M1 2020 512 gb: 666 шт., 5.79% от общей выручки
Macbook Pro 15 M1 2020 512 gb: 1337 шт., 15.98% от общей выручки
Микрофон Rode NT1: 19768 шт., 49.40% от общей выручки
Микрофон Neumann TLM-103: 12 шт., 0.15% от общей выручки
Игровой компьютер ZXC i9-14900kf/4090: 500 шт., 23.85% от общей выручки
Игровой компьютер CXZ i3-6500/1060: 777 шт., 3.04% от общей выручки
